In [6]:
# Nonlinear Least Squares Adjustment of Distance and Angle Observations
#
# This script calculates the coordinates of points P6, P15, and P16
# using nonlinear least squares adjustment based on distance and angle measurements.
# In this version, a weight matrix is included and the dependent angle is removed.

import numpy as np

# Step 1: Define known points (coordinates are in meters)
known_points = {
    'P1': np.array([1000.000, 1558.879]),
    'P2': np.array([1464.895, 1775.091]),
    'P3': np.array([1965.009, 1545.090]),
    'P5': np.array([1000.000, 1000.000])
}

# Step 2: Define initial approximate coordinates for unknown points
initial_coords = {
    'P6': np.array([1269.918, 1259.823]),
    'P15': np.array([2310.045, 1119.938]),
    'P16': np.array([1604.825, 1000.013])
}

# Create initial unknown vector X (6 elements: x_P6, y_P6, x_P15, y_P15, x_P16, y_P16)
initial_X = np.concatenate([initial_coords['P6'], initial_coords['P15'], initial_coords['P16']])
print("Initial unknown vector:", initial_X)

# Step 3: Define observations
# Distance observations (12 values) with their variances
distances = [
    ('P5', 'P1', 560.015, 0.009),
    ('P5', 'P2', 903.793, 0.011),
    ('P5', 'P3', 1108.26, 0.013),
    ('P5', 'P16', 604.996, 0.009),
    ('P16', 'P1', 824.383, 0.011),
    ('P16', 'P2', 787.55, 0.011),
    ('P16', 'P3', 653.161, 0.010),
    ('P2', 'P1', 512.313, 0.009),
    ('P2', 'P3', 550.361, 0.009),
    ('P1', 'P3', 965.106, 0.012),
    ('P16', 'P15', 715.14, 0.010),
    ('P3', 'P15', 547.404, 0.009)
]

# Angle observations (4 values - angle P2-P6-P1 is removed) with their variances
angles = [
    ('P2', 'P6', 'P3', 46 + 57/60 + 55.9/3600, 3/3600),   # 46°57'55.9"
    ('P2', 'P6', 'P16', 107 + 4/60 + 37.3/3600, 3/3600),   # 107°04'37.3"
    ('P2', 'P6', 'P5', 205 + 20/60 + 32.0/3600, 3/3600),   # 205°20'32.0"
    ('P3', 'P16', 'P5', 236 + 33/60 + 11.6/3600, 3/3600)   # 236°33'11.6"
]

# Observation vector L (distances + angles)
L = np.array([d[2] for d in distances] + [a[3] for a in angles])
print("Observation vector L:", L)

# Weight matrix (diagonal of inverse variances)
variances = np.array([d[3]**2 for d in distances] + [a[4]**2 for a in angles])
W = np.diag(1 / variances)
print("Weight matrix (diagonal):", np.diag(W))

# Step 4: Model functions

# Compute Euclidean distance between two points
def compute_distance(point1, point2):
    return np.sqrt(np.sum((point1 - point2) ** 2))

# Compute angle between three points (A-B-C, vertex at B)
def compute_angle(point_A, point_B, point_C):
    vector_BA = point_A - point_B
    vector_BC = point_C - point_B
    angle_BA = np.arctan2(vector_BA[1], vector_BA[0])
    angle_BC = np.arctan2(vector_BC[1], vector_BC[0])
    angle = (angle_BA - angle_BC) * 180 / np.pi
    if angle < 0:
        angle += 360
    return angle

# Compute model vector F(X)
def compute_model(X, known_points, distances, angles):
    points = known_points.copy()
    points['P6'] = X[0:2]
    points['P15'] = X[2:4]
    points['P16'] = X[4:6]
    
    model_distances = [compute_distance(points[p1], points[p2]) for p1, p2, _, _ in distances]
    model_angles = [compute_angle(points[a], points[b], points[c]) for a, b, c, _, _ in angles]
    
    return np.array(model_distances + model_angles)

# Step 5: Compute residual vector (F(X) - L)
def compute_residuals(X, known_points, distances, angles, L):
    F = compute_model(X, known_points, distances, angles)
    return F - L

# Step 6: Compute design (Jacobian) matrix
def compute_jacobian(X, known_points, distances, angles):
    jacobian = np.zeros((len(distances) + len(angles), 6))
    points = known_points.copy()
    points['P6'] = X[0:2]
    points['P15'] = X[2:4]
    points['P16'] = X[4:6]
    
    # Partial derivatives for distances
    for i, (from_pt, to_pt, _, _) in enumerate(distances):
        p1, p2 = points[from_pt], points[to_pt]
        dist = compute_distance(p1, p2)
        if dist == 0:
            continue
        for pt, sign in [(from_pt, 1), (to_pt, -1)]:
            if pt in ['P6', 'P15', 'P16']:
                idx = {'P6': 0, 'P15': 2, 'P16': 4}[pt]
                jacobian[i, idx] += sign * (p1[0] - p2[0]) / dist
                jacobian[i, idx + 1] += sign * (p1[1] - p2[1]) / dist
    
    # Partial derivatives for angles
    for j, (A, B, C, _, _) in enumerate(angles):
        row = len(distances) + j
        idx_B = {'P6': 0, 'P15': 2, 'P16': 4}[B]
        A_pt, B_pt, C_pt = points[A], points[B], points[C]
        
        dx_BA = A_pt[0] - B_pt[0]
        dy_BA = A_pt[1] - B_pt[1]
        dx_BC = C_pt[0] - B_pt[0]
        dy_BC = C_pt[1] - B_pt[1]
        denom_BA = dx_BA**2 + dy_BA**2
        denom_BC = dx_BC**2 + dy_BC**2
        
        dtheta_dxB = (-dy_BA / denom_BA + dy_BC / denom_BC) * 180 / np.pi
        dtheta_dyB = (dx_BA / denom_BA - dx_BC / denom_BC) * 180 / np.pi
        jacobian[row, idx_B] = dtheta_dxB
        jacobian[row, idx_B + 1] = dtheta_dyB
        
        if A in ['P6', 'P15', 'P16']:
            idx_A = {'P6': 0, 'P15': 2, 'P16': 4}[A]
            jacobian[row, idx_A] = (dy_BA / denom_BA) * 180 / np.pi
            jacobian[row, idx_A + 1] = (-dx_BA / denom_BA) * 180 / np.pi
        if C in ['P6', 'P15', 'P16']:
            idx_C = {'P6': 0, 'P15': 2, 'P16': 4}[C]
            jacobian[row, idx_C] = (-dy_BC / denom_BC) * 180 / np.pi
            jacobian[row, idx_C + 1] = (dx_BC / denom_BC) * 180 / np.pi
            
    return jacobian

# Step 7: Gauss-Newton method with weight matrix
def nonlinear_least_squares(initial_X, known_points, distances, angles, L, W, max_iterations=100, tolerance=1e-6):
    X = initial_X.copy()
    for iteration in range(max_iterations):
        residuals = compute_residuals(X, known_points, distances, angles, L)
        jacobian = compute_jacobian(X, known_points, distances, angles)
        
        # Weighted normal equation components
        JtWJ = jacobian.T @ W @ jacobian
        JtWR = jacobian.T @ W @ residuals
        
        # Compute condition number of normal matrix
        cond_number = np.linalg.cond(JtWJ)
        print(f"Iteration {iteration + 1}: Condition number of JᵀWJ = {cond_number:.2e}")
        
        # Solve using pseudo-inverse
        delta = -np.linalg.pinv(JtWJ) @ JtWR
        X += delta
        
        if np.linalg.norm(delta) < tolerance:
            print("Convergence achieved.")
            break
            
    return X

# Final call to perform the adjustment
adjusted_X = nonlinear_least_squares(initial_X, known_points, distances, angles, L, W)
print("Adjusted coordinates (P6, P15, P16):", adjusted_X)

# Step 9: Display results
print("Final coordinates:")
print("P6:", final_X[0:2])
print("P15:", final_X[2:4])
print("P16:", final_X[4:6])

# Step 10: Validation
final_model = compute_model(final_X, known_points, distances, angles)
print("Computed values (distances and angles):", final_model)
print("Observed values:", L)
print("Errors:", final_model - L)


Initial unknown vector: [1269.918 1259.823 2310.045 1119.938 1604.825 1000.013]
Observation vector L: [ 560.015       903.793      1108.26        604.996       824.383
  787.55        653.161       512.313       550.361       965.106
  715.14        547.404        46.96552778  107.07702778  205.34222222
  236.55322222]
Weight matrix (diagonal): [  12345.67901235    8264.46280992    5917.15976331   12345.67901235
    8264.46280992    8264.46280992   10000.           12345.67901235
   12345.67901235    6944.44444444   10000.           12345.67901235
 1440000.         1440000.         1440000.         1440000.        ]
Iteration 1: Condition number of JᵀWJ = 3.25e+01
Iteration 2: Condition number of JᵀWJ = 3.25e+01
Iteration 3: Condition number of JᵀWJ = 3.24e+01
Iteration 4: Condition number of JᵀWJ = 3.24e+01
Iteration 5: Condition number of JᵀWJ = 3.24e+01
Iteration 6: Condition number of JᵀWJ = 3.23e+01
Iteration 7: Condition number of JᵀWJ = 3.21e+01
Iteration 8: Condition number of 